# Amazon SageMaker で US の国勢調査データを分析する

このノートブックの内容は、こちらの [ブログ](https://aws.amazon.com/jp/blogs/news/analyze-us-census-data-for-population-segmentation-using-amazon-sagemaker/) の内容に沿っています。


## Introduction

米国では、2018年の中間選挙が近づいているため、人々は投票するにあたって詳細情報を求めています。このノートブックの例では、有権者を理解するタスクを科学的に実施するために、機械学習（ML）を適用する方法を検討します。

通常、機械学習を使用する場合、一般的なユースケースはラベル付きデータから開始します。たとえば、使用年数やモデル番号などのデバイスの属性に基づいて、故障の可能性を予測できます。特定の結果を予測するための教師データまたはガイダンスがあるため、この教師あり学習と呼びます。

ただし、現実の世界では、予測する特定の結果がなく、クリーンなラベルを定義するのが難しい大きなデータセットがしばしばあります。正しい結果を予測することを正確に特定することは困難です。このタイプのユースケースは、多くの場合試行錯誤が伴います。データセットの構成と、どのような自然なパターンが存在するかを理解する必要があります。このタイプのユースケースは、教師なし学習として知られています。ひとつの例としては、一連の属性に基づいた個人のグルーピングがあります。

このノートブックで探るユースケースは、人口のセグメンテーションです。 https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml では、さまざまな米国の郡の人口統計に関する米国国勢調査の匿名データを公開しています。 （この製品は国勢調査局のAPIを使用していますが、国勢調査局によって承認または認定されたものではないことに注意してください。）郡が属するクラスターを活用して、選挙キャンペーンの計画を立てることができます。たとえば、そのグループに響くようなメッセージを強調することにより、類似した郡のグループにコンタクトする方法がわかります。より一般的には、この手法は顧客またはユーザーのセグメンテーションのビジネスに適用して、ターゲットを絞ったマーケティングキャンペーンを作成できます。このタイプの分析には、CA-Fresno郡とAZ-Yuma郡がグループ化されているなど、一見明らかではない類似性を明らかにする機能があります。直感的には、人口規模や人種構成などの一般的に検討される属性とは異なりますが、雇用タイプの組み合わせなどの軸に沿って見た場合よりも近く見えるのではと思います。


この演習には2つの目標があります。

1. Amazon SageMakerを使用して、PCAおよびKmeansモデリング手法などの教師なし学習を使用したデータサイエンスワークフローを体験します。

1. ユーザーがAmazon SageMaker内で構築された基礎となるモデルにアクセスして、有用なモデル属性を抽出する方法を示します。 教師なし学習から結論を引き出すことは困難な場合が多いため、PCAとKmeansのモデルにアクセスできることは、モデルを使用して単純に予測を生成することよりもさらに重要になります。

このノートブックは、大きく4つのステップに分かれています:

1. [Loading the data from Amazon S3](#Step-1:-Loading-the-data-from-Amazon-S3)
2. [Exploratory data analysis (EDA) - Data cleaning and exploration](#Step-2:-Exploratory-data-analysis-EDA---Data-cleaning-and-exploration)
   1. [Cleaning the data](#a.-Cleaning-the-data)
   2. [Visualizing the data](#b.-Visualizing-the-data)
   3. [Feature engineering](#c.-Feature-engineering)
3. [Data modelling](#Step-3:-Data-modelling)
   1. [Dimensionality reduction](#a.-Dimensionality-reduction)
   2. [Accessing the PCA model attributes](#b.-Accessing-the-PCA-model-attributes)
   3. [Deploying the PCA model](#c.-Deploying-the-PCA-model)
   4. [Population segmentation using unsupervised clustering](#d.-Population-segmentation-using-unsupervised-clustering)
4. [Drawing conclusions from our modelling](#Step-4:-Drawing-conclusions-from-our-modelling)
    1. [Accessing the KMeans model attributes](#a.-Accessing-the-KMeans-model-attributes)

## Step 1: Loading the data from Amazon S3

データセットをAmazon S3バケットからAmazon SageMakerノートブックにロードする必要があります。

最初に、関連するライブラリをAmazon SageMakerノートブックにインポートします。

In [ ]:
import os
import boto3
import io
import sagemaker

%matplotlib inline 

import pandas as pd
import numpy as np
import mxnet as mx
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
matplotlib.style.use('ggplot')
import pickle, gzip, urllib, json
import csv

Amazon SageMakerは、Amazon S3とシームレスに統合します。 ノートブック作成の最初のステップで、ノートブックに「AmazonSageMakerFullAccess」ロールを指定しました。 これにより、このノートブックに、名前に「sagemaker」が含まれるこのAWSアカウントのAmazon S3バケットにアクセスする許可が与えられます。

get_execution_role関数は、ノートブックインスタンスを作成したときに作成したIAMロールを取得します。

In [ ]:
from sagemaker import get_execution_role
role = get_execution_role()

ロールの名前が AmazonSageMaker-ExecutionRole　であることがわかります。

In [ ]:
role

#### Loading the dataset
あらかじめ、アクセス可能なパブリックS3バケットにデータをダウンロードして保存しておきました。 Python SDK、Boto3クライアントを使用してAWSのサービスとやり取りできます。

まず、クライアントを起動します。

In [ ]:
s3_client = boto3.client('s3')
data_bucket_name='aws-ml-blog-sagemaker-census-segmentation'

バケット内に含まれるオブジェクトのリストを取得します。 バケットに「Census_Data_for_SageMaker.csv」というファイルが1つあることがわかります。

In [ ]:
obj_list=s3_client.list_objects(Bucket=data_bucket_name)
file=[]
for contents in obj_list['Contents']:
    file.append(contents['Key'])
print(file)

In [ ]:
file_data=file[0]

バケット内のCSVファイルからデータを取得します。

In [ ]:
response = s3_client.get_object(Bucket=data_bucket_name, Key=file_data)
response_body = response["Body"].read()
counties = pd.read_csv(io.BytesIO(response_body), header=0, delimiter=",", low_memory=False) 

データの最初の5行は次のようになります。

In [ ]:
counties.head()

## Step 2: Exploratory data analysis *EDA* - Data cleaning and exploration
### a. Cleaning the data
ノートブックインスタンスを使用して簡単なデータクリーニングと処理を直接実行できます。

どのくらいのデータがあるでしょうか？

37列、3220行のデータのようです。

In [ ]:
counties.shape

欠損値があるデータを削除して、分析を容易にします。 2つの行が失われたことがわかります。現在、データには3218行あります。

In [ ]:
counties.dropna(inplace=True)
counties.shape

分析に不要な調査番号は削除し、州や郡のカテゴリーデータはひとつにまとめ、数値データはそのままにします。

これで、インデックスとして「州郡」を設定でき、残りの数値的データは一意の属性を表すことになります。

In [ ]:
counties.index=counties['State'] + "-" + counties['County']
counties.head()
drop=["CensusId" , "State" , "County"]
counties.drop(drop, axis=1, inplace=True)
counties.head()

### b. Visualizing the data
これで、数値データとカテゴリデータが混在したデータセットができました。 一部の数値データを視覚化し、分布がどのように見えるかを確認します。

In [ ]:
import seaborn as sns

for a in ['Professional', 'Service', 'Office']:
    ax=plt.subplots(figsize=(6,3))
    ax=sns.distplot(counties[a])
    title="Histogram of " + a
    ax.set_title(title, fontsize=12)
    plt.show()

たとえば、上記の図から、専門職、サービス職、またはオフィス職の労働者の割合の分布を観察できます。 ヒストグラムを表示すると、平均や歪度など、これらの機能の特性を視覚的に示すことができます。 たとえば、専門職の分布から、典型的な郡には約25〜30％の専門職がおり、一部の郡では専門職が80％近くに達していることがわかります。

### c. Feature engineering

**Data Scaling**- 距離ベースの分析手法を使用して異なる特徴間の相対距離を比較できるように、数値データのスケーリングを標準化する必要があります。 minmaxscalerを使用して、数値列を0から1の間になるように変換できます。

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
counties_scaled=pd.DataFrame(scaler.fit_transform(counties))
counties_scaled.columns=counties.columns
counties_scaled.index=counties.index

すべての数値列の最小値が0、最大値が1になっていることがわかります。

In [ ]:
counties_scaled.describe()

## Step 3: Data modelling
### a. Dimensionality reduction

主成分分析（PCA）を使用してデータの次元を減らします。この方法は、データ行列を互いに直交する特徴に分解します。結果として得られた直交する特徴量は、元の特徴量の線形結合です。この方法は、多くの特徴量を吟味して類似または冗長な特徴量を組み合わせて、新しい小さな特徴セットを形成するものと考えることができます。

Amazon SageMaker に組み込まれているPCAアルゴリズムを使用して、次元を減らすことができます。

まず、SageMakerのPCAモデルをインポートして呼び出します。次に、モデルのさまざまなパラメーターを指定します。これらは、モデルの学習中に使用するインスタンスの数、使用するインスタンスのタイプなどのリソース構成を設定するパラメーターです。さらに、PCAを実行するときに使用するコンポーネントの数など、モデル計算のハイパーパラメーターも設定することもできます。 PCAモデルに関するドキュメントは、http://sagemaker.readthedocs.io/en/latest/pca.html　にあります。

SageMakerがモデルパラメータを保存するためのバケット名を指定できます。バケットはこのノートブックと同じリージョンにある必要があることに注意してください。

In [ ]:
bucket='20190604-obc'

In [ ]:
from sagemaker import PCA
num_components=33

pca_SM = PCA(role=role,
             train_instance_count=1,
             train_instance_type='ml.c4.xlarge',
             output_path='s3://'+ bucket +'/counties/',
             num_components=num_components)

次に、DataFrameからnumpy配列を抽出し、明示的に`float32`にキャストすることにより、Amazon SageMakerで使うためのデータを準備します。

In [ ]:
train_data = counties_scaled.values.astype('float32')

Amazon SageMaker のPCAモデルのrecord_set関数はnumpy配列を、トレーニングする入力データに必要な形式であるprotobuf recordIO 形式に変換します。 これは、すべてのAmazon SageMakerビルトインアルゴリズムの要件です。 このデータ形式は、sklearnなどで実装された同じモデルと比較して、より大きなデータセットに対して、Amazon SageMakerがモデルの学習をより速く実行できる理由の1つです。

PCAモデルでfit関数を呼び出し、学習データを渡します。これにより、学習用インスタンスまたはクラスターが起動し学習ジョブが実行されます。

In [ ]:
%%time
pca_SM.fit(pca_SM.record_set(train_data))

### b. Accessing the PCA model attributes

モデルの学習が終わったら、モデルパラメーターにもアクセスできます。

学習ジョブが完了したので、Amazon SageMakerコンソールの[トレーニング]サブセクションの[ジョブ]でジョブの詳細を確認することができます。

学習済みのモデルは、Amazon S3に保存されます。 これは、Amazon SageMakerを使用して学習済みモデルをデプロイするために使用されます。 Amazon SageMakerのアルゴリズムの多くは機械学習フレームワークにMXNetを使用しているため、学習済みモデルはndarrayとして保存されます。 モデルは、学習ジョブを起動する際に指定した出力パスの下の`<training_job_name> /output/model.tar.gz`に保存されます。これは、GNU zip（gzip）圧縮で圧縮されたTARアーカイブファイルです。

In [ ]:
job_name = pca_SM.latest_training_job.name
model_key = "counties/" + job_name + "/output/model.tar.gz"

boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')

モデルが解凍された後、MXNetを使用して ndarray をロードできます。

In [ ]:
import mxnet as mx
pca_model_params = mx.ndarray.load('model_algo-1')

**PCAモデルのパラメタには3つのグループがあります。**

**mean**：データセットの平均の行。この値は、subtract_mean ハイパーパラメーターが true に設定されている場合にのみ表示され、モデルは PCA 投影を適用する前にデータが平均ゼロの行になるようにシフトします。

**v**：各行が固有ベクトルである主成分。最も重要でないものから、最も重要なものへという順序になります。（sklearn PCAモデルの「components_」と同じ）。

**s**：特異値。昇順に並べ替えられます。これらが、各固有ベクトルの重要性を決定します。

説明された分散比 ~= square(s) / sum(square(s))

必要に応じて正確な説明付き分散比ベクトルを計算するには、元のデータの二乗和を保存し（Nと呼ぶ）、説明付き分散比= square（s）/ Nを計算するだけです。


In [ ]:
s=pd.DataFrame(pca_model_params['s'].asnumpy())
v=pd.DataFrame(pca_model_params['v'].asnumpy())

これで、保持したい最大のn個の成分によって説明される寄与率を計算できます。 この例では、上位5つのコンポーネントを取り上げます。

33のコンポーネントのうち最大の5つのコンポーネント（28から32の5つ）の累積寄与率が〜72％であることがわかります。

In [ ]:
s.iloc[28:,:].apply(lambda x: x*x).sum()/s.apply(lambda x: x*x).sum()

上位5つのコンポーネントを使用することを決めたら、元のsおよびvから最大の5つのコンポーネントのみを取得します。

In [ ]:
s_5=s.iloc[28:,:]
v_5=v.iloc[:,28:]
v_5.columns=[0,1,2,3,4]

コンポーネントに含まれる元の特徴の重み付けに基づいて、各PCAコンポーネントの構成を調べることができます。 たとえば、次のコードは最初のコンポーネントを示しています。 このコンポーネントは、貧困と失業率が高く、一人当たりの所得と世帯所得が低く、ヒスパニック/黒人人口が多く、白人人口が少ない郡の属性を表していることがわかります。

これはv_5 [4]またはv_5のコンポーネントのリストの最後のコンポーネントですが、コンポーネントは最小から最大の順に並べられているため、実際には最大のコンポーネントです。 したがって、v_5 [0]は最小のコンポーネントになります。 同様に、component_numの値を変更して、各コンポーネントの構成を循環させます。

In [ ]:
component_num=1

first_comp = v_5[5-component_num]
comps = pd.DataFrame(list(zip(first_comp, counties_scaled.columns)), columns=['weights', 'features'])
comps['abs_weights']=comps['weights'].apply(lambda x: np.abs(x))
ax=sns.barplot(data=comps.sort_values('abs_weights', ascending=False).head(10), x="weights", y="features", palette="Blues_d")
ax.set_title("PCA Component Makeup: #" + str(component_num))
plt.show()

同様に、各PCAコンポーネントの構成を調べて調べ、各コンポーネントの重要な肯定的および否定的な属性が何であるかを理解しようとすることができます。 次のコードはコンポーネントに名前を付けていますが、各コンポーネントの独自の構成を理解できるように、自由に変更してください。

In [ ]:
PCA_list=['comp_1', 'comp_2', 'comp_3', 'comp_4', 'comp_5']

#PCA_list=["Poverty/Unemployment", "Self Employment/Public Workers", "High Income/Professional & Office Workers", \
#         "Black/Native Am Populations & Public/Professional Workers", "Construction & Commuters"]

### c. Deploying the PCA model

学習済みのモデルを使ってエンドポイントを起動し、それを使用して予測を行うことができます。 エンドポイントは、指定したinstance_typeでホストされます。

In [ ]:
%%time
pca_predictor = pca_SM.deploy(initial_instance_count=1, 
                                 instance_type='ml.t2.medium')

元のデータセットをモデルに渡して、作成したモデルを使用してデータを変換することもできます。 次に、最大の5つのコンポーネントを取得できます。これにより、データの次元が34から5に減少します。

In [ ]:
%%time
result = pca_predictor.predict(train_data)

In [ ]:
result[10]

In [ ]:
counties_transformed=pd.DataFrame()
for a in result:
    b=a.label['projection'].float32_tensor.values
    counties_transformed=counties_transformed.append([list(b)])
counties_transformed.index=counties_scaled.index
counties_transformed=counties_transformed.iloc[:,28:]
counties_transformed.columns=PCA_list

これで、以前に分析した5つの主要コンポーネントによって各郡が記述されたデータセットが作成されました。 これら5つのコンポーネントはそれぞれ、元の特徴空間の線形結合です。 前に示したコンポーネントの構成を分析することにより、これら5つのコンポーネントのそれぞれを解釈できます。

In [ ]:
counties_transformed.head(10)

### d. Population segmentation using unsupervised clustering

次に、Kmeansアルゴリズムを使用して、作成した5つのPCA属性で郡の人口をセグメント化します。 Kmeansは、属性に基づいて同様の郡のクラスターを識別するクラスタリングアルゴリズムです。 元のデータセットには〜3000の郡と34の属性があるため、大きなフィーチャスペースにより、郡を効果的にクラスタ化することが困難になった可能性があります。 代わりに、特徴空間を5つのPCAコンポーネントに削減し、この変換されたデータセットでクラスター化します。

In [ ]:
train_data = counties_transformed.values.astype('float32')

まず、PCAモデルで行ったようにKMeansモデルのハイパーパラメーターを呼び出して定義します。 Kmeansアルゴリズムにより、ユーザーは特定するクラスターの数を指定できます。 この例では、データセットから上位7つのクラスターを見つけてみましょう。

In [ ]:
from sagemaker import KMeans

num_clusters = 7
kmeans = KMeans(role=role,
                train_instance_count=1,
                train_instance_type='ml.c4.xlarge',
                output_path='s3://'+ bucket +'/counties/',              
                k=num_clusters)

次に、トレーニングデータでモデルをトレーニングします。

In [ ]:
%%time
kmeans.fit(kmeans.record_set(train_data))

次に、モデルを展開し、元のトレーニングセットを渡して、各エントリのラベルを取得します。 これにより、各郡が属するクラスターがわかります。

In [ ]:
%%time
kmeans_predictor = kmeans.deploy(initial_instance_count=1, 
                                 instance_type='ml.t2.medium')

In [ ]:
%%time
result=kmeans_predictor.predict(train_data)

クラスターカウントの内訳とクラスターの分布を確認できます。

In [ ]:
cluster_labels = [r.label['closest_cluster'].float32_tensor.values[0] for r in result]

In [ ]:
pd.DataFrame(cluster_labels)[0].value_counts()

In [ ]:
ax=plt.subplots(figsize=(6,3))
ax=sns.distplot(cluster_labels, kde=False)
title="Histogram of Cluster Counts"
ax.set_title(title, fontsize=12)
plt.show()

ただし、説明可能性を改善するには、基になるモデルにアクセスしてクラスターの中心を取得する必要があります。 これらのセンターは、各クラスターを特徴付ける機能の説明に役立ちます。

## Step 4: Drawing conclusions from our modelling

モデリングの結果を説明することは、分析を活用する上で重要なステップです。 PCAとKmeans、およびAmazon SageMakerトレーニング済みモデル内のモデル属性に含まれる情報を組み合わせることにより、データに基づいて具体的な結論を形成できます。

### a. Accessing the KMeans model attributes

最初に、kmeansモデルが保存されているバケットに移動して抽出します。

In [ ]:
job_name = kmeans.latest_training_job.name
model_key = "counties/" + job_name + "/output/model.tar.gz"

boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')

In [ ]:
Kmeans_model_params = mx.ndarray.load('model_algo-1')


**KMeansモデルに含まれるモデルパラメーターのセットが1つあります。**

**クラスター重心位置**：Kmeansアルゴリズムによって識別された各クラスターの中心の位置。 クラスターの位置は、変換されたPCAデータをモデルに渡したため、5つのコンポーネントを持つPCA変換空間で与えられます。

In [ ]:
cluster_centroids=pd.DataFrame(Kmeans_model_params[0].asnumpy())
cluster_centroids.columns=counties_transformed.columns

In [ ]:
cluster_centroids

変換された特徴空間における重心とその位置のヒートマップをプロットできます。 これにより、各クラスターを定義する特性に関する洞察が得られます。 多くの場合、教師なし学習では、結果を解釈するのは困難です。 これは、PCAとクラスタリング手法の結果を一緒に利用する1つの方法です。 各PCAコンポーネントの構成を調べることができたため、各重心が以前に解釈したPCAコンポーネントに関して何を表すかを理解できます。

たとえば、他のクラスターと比較して、クラスター1の「建設と通勤者」属性の値が最も高く、「自営業/公務員」属性の値が最も低いことがわかります。 同様に、クラスター4は、「建設＆通勤者」、「高収入/専門職とオフィスワーカー」、「自営業/公務員」の値が高くなっています。

In [ ]:
plt.figure(figsize = (16, 6))
ax = sns.heatmap(cluster_centroids.T, cmap = 'YlGnBu')
ax.set_xlabel("Cluster")
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
ax.set_title("Attribute Value by Centroid")
plt.show()

また、クラスターラベルを個々の郡にマップし、どの郡が自然にグループ化されたかを調べることもできます。

In [ ]:
counties_transformed['labels']=list(map(int, cluster_labels))
counties_transformed.head()

これで、たとえばクラスター1のように、クラスターの1つをより詳細に調べることができます。 重心の位置をざっと見てみると、「Construction＆Commuters」属性の値が最も高いことがわかります。 これで、どの郡がその説明に適合するかを確認できます。

In [ ]:
cluster=counties_transformed[counties_transformed['labels']==1]
cluster.head(5)

## Conclusion

教師なし学習のデータサイエンスワークフロー、特にPCAを使用して次元数を削減した後、KMeansを使用してデータセットをクラスター化するワークフローについて説明しました。 Amazon SageMaker内で作成された基礎となるモデルにアクセスすることで、モデリングの説明可能性を改善し、実用的な結論を導き出すことができました。 これらの手法を使用して、米国のさまざまな郡の本質的な特性をよりよく理解し、それに応じて選挙人をグループ分けすることができました。

**エンドポイントは停止させるまで課金されるため、利用が終わったらエンドポイントを削除しましょう。**

In [ ]:
sagemaker.Session().delete_endpoint(pca_predictor.endpoint)

In [ ]:
sagemaker.Session().delete_endpoint(kmeans_predictor.endpoint)